# Connect N games (gomoku, connect 6)
This is generalized connect 6 [games](http://en.wikipedia.org/wiki/Connect6) invented by professor [吳毅成](https://en.wikipedia.org/wiki/I-Chen_Wu). A game is described as a tuple of $G(m, n, c, i, j)$. Black and white players take turns to play on a $m \cdot n$ go-like board, black plays initial $i$ moves and afterwards each player alternately plays $j$ pieces. The one who connects $c$ pieces horizontally, vertically or diagonally wins.

A gomoku game can be described as G(15, 15, 5, 1, 1) and a connect 6 games can be described as G(19, 19, 6, 1, 2).

In [8]:
from math import pi
from ipycanvas import Canvas
from ipywidgets import widgets
from IPython.display import display, HTML
from enum import Enum
import numpy as np

DrawBoard class supports interactive game playing using [ipycanvas](https://github.com/martinRenou/ipycanvas). An abstract game board is represented as a dimension $m, n$ numpy array. Where each cell is represented as either,
    1. empty 0
    2. black -1
    3. white +1
This way, the black pieces can be find out by using ```board < 0``` and white by ```board > 0```.

In [9]:
class PieceColor(Enum):
    empty = 0
    black = -1
    white = +1

In [10]:
class DrawBoard:
    def __init__(
        self,
        canvas,
        width,
        height,
        offsetx = 2,
        offsety = 2,
        unit = 20
    ):
        """
        Initialize an empty drawing board.
        Args:
            canvas: An ipycanvas object, at least (width + offsetx) * unit wide
                    and (height + offsety) * unit tall.
            width, height: dimension of the board
            offsetx, offsety: position of the top-left corner.
            unit: cell size.
        """
        self.fonts = [
            (36, "36px serif"),
            (24, "24px serif"),
            (16, "16px serif"),
            (12, "12px serif"),
            (8,  "8px serif"),
            (0,  "")
        ]
        self.canvas = canvas
        self.unit = unit
        self.offsetx = offsetx
        self.offsety = offsety
        self.width = width
        self.height = height

        
    def draw(self):
        '''
        Draw an empty board
        '''
        self.canvas.stroke_style = "black"
        self.canvas.begin_path()
        for x in range(self.offsetx, self.offsetx + self.width):
            self.canvas.move_to(x * self.unit, self.offsety * self.unit)
            self.canvas.line_to(x * self.unit, (self.offsety + self.height - 1) * self.unit)
        for y in range(self.offsety, self.offsety + self.height):
            self.canvas.move_to(self.offsetx * self.unit, y * self.unit)
            self.canvas.line_to((self.offsetx + self.width - 1) * self.unit, y * self.unit)
        self.canvas.stroke()
        
    def piece(self, x, y, color, nmove = None):
        '''
        draw a piece at coordinate x, y
        Args:
            x, y: coordinates of the stone.
            color: color of the stone, if color is PieceColor.empty, clear the cell
            nmove: A number, if not None, label the stone with nmove
        '''
        px, py = (x + self.offsetx) * self.unit, (y + self.offsety) * self.unit
        if color == PieceColor.empty:
            self.canvas.clear_rect(
                px - (self.unit + 1) // 2,
                py - (self.unit + 1) // 2,
                (self.unit + 1) // 2 * 2,
                (self.unit + 1) // 2 * 2
            )
            return
        self.canvas.stroke_style = "black"
        self.canvas.fill_style = "black" if color == PieceColor.black else "white"
        radius = self.unit // 2 - 1
        self.canvas.stroke_arc(px, py, radius, 0, 2 * pi)
        self.canvas.fill_arc(px, py, radius, 0, 2 * pi)
        font_offset = 0
        # draw label
        if nmove != None:
            label = str(nmove)
            for threshold, font in self.fonts:
                if self.unit > len(label) * threshold:
                    self.canvas.font = font
                    font_offset = threshold
                    break
            self.canvas.fill_style = "white" if color == PieceColor.black else "black"
            self.canvas.fill_text(
                label,
                px - int(font_offset / 2 * len(label) / 2),
                py + font_offset // 3,
                self.unit
            )

    def mark(self, connected_pieces, color):
        '''
        Mark connected_pieces with color. Used at the end of the game to mark the
        pieces that form the straight line.
        Args:
            connected_pieces: A list of pieces that form the straight line (winning condition)
            color: color of the pieces.
        '''
        self.canvas.fill_style = "black" if color == PieceColor.white else "white"
        for x, y in connected_pieces:
            px, py = (x + self.offsetx) * self.unit, (y + self.offsety) * self.unit
            self.canvas.fill_rect(
                px - self.unit // 6,
                py - self.unit // 6,
                self.unit // 3,
                self.unit // 3
            )

    def where(self, x, y):
        '''
        returns position on the board ((0, width), (0, height)), giving absolute
        ipycanvas coordinates (mouse position)
        Args:
            x, y: mouse position
        Returns:
            None, if the mouse position is out of board.
            Position on the board ((0, width), (0, height))
        '''
        x = x - self.offsetx * self.unit
        y = y - self.offsety * self.unit
        x = int((x + self.unit // 2) // self.unit)
        y = int((y + self.unit // 2) // self.unit)
        if x < 0 or x >= self.width:
            return None
        if y < 0 or y >= self.height:
            return None
        return x, y


Game is the implementation of connectN game.

In [20]:
# console
out = widgets.Output()

class Game:
    def __init__(
        self,
        M,
        N,
        C,
        I,
        J,
        size = (500, 500),
    ):
        '''
        Initialize a game of $G(M, N, C, I, J)
        Args:
            M, N: width and height of the board.
            C: Number of stones connected to win.
            I: Number of initial moves.
            J: Number of moves by a player before changing side.
            size: canvas size (for interactive games).
        '''
        self.canvas = Canvas(size = size) if size else None
        self.width, self.height = M, N 
        self.win = C
        self.move = J
        self.imove = I
        self.nmoves = -self.imove
        self.finish = PieceColor.empty
        self.matrix = np.zeros((self.width + 2, self.height + 2), dtype=np.int)
        self.neighbors = [(1, 1), (1, -1), (1, 0), (0, 1)]
        assert(self.width >= self.win and self.height >= self.win)
        if self.canvas:
            unit = self.width if self.width < self.height else self.height
            canvas_size = size[0] if size[0] < size[1] else size[1]
            unit = canvas_size // (unit + 4)
            self.board = DrawBoard(
                self.canvas,
                self.width,
                self.height,
                2,
                2,
                unit
            )
            self.board.draw()
            for x in range(self.imove):
                self.board.piece(x, -1, PieceColor.black)
            self.canvas.on_mouse_down(lambda x, y: self.do_move(x, y))

    def check_connection(self, x, y, color):
        '''
        Check win condition.
        Args:
            x, y: position of the last move
            color: color of the last move
        Returns:
            Array of postion of stones that form a winning condition.
            If the array is empty, the game is not finished.
        '''
        for direction, (dx, dy) in enumerate(self.neighbors):
            connected_pieces = [(x, y)]
            for s in -1, 1:
                for i in range(1, self.win):
                    nx, ny = x + dx * i * s, y + dy * i * s
                    if self.matrix[nx, ny] == color:
                        connected_pieces.append((nx, ny))
                    else:
                        break
            if len(connected_pieces) >= self.win:
                return connected_pieces
        return []

    @out.capture()
    def do_move(self, x, y):
        '''
        One party makes a move, updates state, check winning condition and set self.finish
        to the player who wins the game.
        Args:
            x, y: position of the piece.
        '''
        if self.finish != PieceColor.empty:
            return
        position = self.board.where(x, y)
        if position == None:
            return
        px, py = position
        mx, my = px + 1, py + 1
        if np.sum(self.matrix[mx, my]) != PieceColor.empty.value:
            return
        nmoves = self.nmoves // self.move if self.nmoves >= 0 else -1
        color = PieceColor.white if nmoves % 2 == 0 else PieceColor.black
        self.matrix[mx, my] = color.value
        self.nmoves += 1
        if self.canvas:
            self.board.piece(px, py, color, self.nmoves + self.imove)
            nmoves = self.nmoves // self.move if self.nmoves >= 0 else -1
            color_next = PieceColor.white if nmoves % 2 == 0 else PieceColor.black
            pieces = -self.nmoves if self.nmoves < 0 else self.move - self.nmoves % self.move
            for i in range(self.imove if self.imove > self.move else self.move):
                if i >= pieces:
                    self.board.piece(i, -1, PieceColor.empty)
                else:
                    self.board.piece(i, -1, color_next)
        connected_pieces = self.check_connection(mx, my, color.value)
        if connected_pieces:
            connected_pieces = [(x[0] - 1, x[1] - 1) for x in connected_pieces]
            self.finish = color
            if self.canvas:
                self.board.mark(connected_pieces, color)
display(out)

Output()

This is sample interactive game $G(19, 19, 6, 5, 3)$. Played on 19 x 19 board, and first player (black) makes 5 initial moves and each player makes 3 moves alternately after. The one who connects 5 stones in a line wins.

In [22]:
game = Game(19, 19, 6, 5, 3, (600, 600))
game.canvas

Canvas(layout=Layout(height='600px', width='600px'), size=(600, 600))